In [106]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [107]:
endpoint = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

res = requests.get(endpoint)

soup = BeautifulSoup(res.text)



In [108]:
us_state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

In [109]:
# def get_all():
#     districts = ['01', '02', ...] 
#     for state in us_state_abbreviations:
#         endpoint = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{districts}&spec=N'

#         res = requests.get(endpoint)

#         soup = BeautifulSoup(res.text)

#         soup.find()

In [110]:
candidate_elements = soup.find_all('h2')

# Placeholder for our extracted data

for element in candidate_elements:
    text = element.get_text(strip=True)
    print(text)

We follow the money.Youmake it possible.
We follow the money. You make it happen.
Mark Green (R) •Incumbent•Winner(69.9% of vote)
Kiran Sreepada (D)(27.3% of vote)
Ronald Brown (I)(2.2% of vote)
Scott Vieira Jr (I)(0.6% of vote)
We follow the money.Youmake it possible.
Count Cash & Make Change


In [111]:
# Parsing HTML
soup = BeautifulSoup(res.text)

pattern_candidate = re.compile(r"(.+?) \((R|D|I)\)( •Incumbent•Winner)?(\(([\d.]+)% of vote\))?")

# Placeholder for extracted data
data = []

# Extract candidate data
for element in soup.find_all('h2'):
    text = element.get_text(strip=True)
    match = pattern_candidate.match(text)
    if match:
        name, party, incumbent_winner, _, vote_percentage = match.groups()
        incumbent = incumbent_winner is not None
        winner = incumbent  # Assumes if they are incumbent they are also the winner
        data.append([name, party, incumbent, winner, vote_percentage, None, None])

# Extract financial data
financial_elements = soup.find_all('tbody')
for idx, financial_element in enumerate(financial_elements):

    try:
        print(financial_element.prettify())

        # Check if this tbody has a 'Members--number' class indicating it's financial data
        if financial_element.find(class_="Members--number"):
            raised_element = financial_element.find_all(string="Raised:")
            spent_element = financial_element.find_all(string="Spent:")
        
            if raised_element:
                raised = raised_element[0].find_next(class_="Members--number").get_text(strip=True)
                raised = raised.replace("$", "").replace(",", "")
            else:
                raised = None

            if spent_element:
                spent = spent_element[0].find_next(class_="Members--number").get_text(strip=True)
                spent = spent.replace("$", "").replace(",", "")
            else:
                spent = None
            
            # Add financial data to the corresponding candidate
            if idx < len(data):
                data[idx].extend([raised, spent])
        else:
            print(f"Skipping financial element {idx} as it doesn't appear to contain financial data")
    except Exception as e:
        print(f"An error occurred while processing financial element {idx}: {str(e)}")

# Creating DataFrame
df = pd.DataFrame(data, columns=["Name", "Party", "Incumbent", "Winner", "Vote Percentage", "Raised", "Spent"])
print(df)


<tbody>
 <tr>
  <td>
   Small Individual Contributions (≤ $200)
  </td>
  <td class="number">
   $0
  </td>
  <td class="number">
   0.00%
  </td>
 </tr>
 <tr>
  <td>
   Large Individual Contributions
  </td>
  <td class="number">
   $819,151
  </td>
  <td class="number">
   68.55%
  </td>
 </tr>
 <tr>
  <td>
   PAC Contributions*
  </td>
  <td class="number">
   $171,900
  </td>
  <td class="number">
   14.39%
  </td>
 </tr>
 <tr>
  <td>
   Candidate self-financing
  </td>
  <td class="number">
   $0
  </td>
  <td class="number">
   0.00%
  </td>
 </tr>
 <tr>
  <td>
   Other
  </td>
  <td class="number">
   $203,909
  </td>
  <td class="number">
   17.06%
  </td>
 </tr>
</tbody>

Skipping financial element 0 as it doesn't appear to contain financial data
<tbody>
 <tr>
  <td nowrap="">
   Full Disclosure
  </td>
  <td class="number">
   $1,061,024
  </td>
  <td class="number">
   98.64%
  </td>
 </tr>
 <tr>
  <td nowrap="">
   Incomplete Disclosure
  </td>
  <td class="number">
   $2,6